In [1]:
# Currency Dynamically Changes Everyday, We will use tool so that in real time tool will convert the INR to Some Other Currency or Vice-versa.

In [41]:
# We will use Exchange Rate API
from langchain_core.tools import tool
from langchain_groq import ChatGroq
import requests, json
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import os
from dotenv import load_dotenv
load_dotenv()

exchange_rate_api_key = str(os.getenv("EXCHANGE_RATE_API_KEY"))

In [42]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """ This Function Fetches the Currency Conersion Factor Between the base currency and Target Currency."""

    url = f"https://v6.exchangerate-api.com/v6/{exchange_rate_api_key}/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given a Currency Conversion Rate this function calculates the target curency value from a given base currency value"""

    return base_currency_value * conversion_rate

In [21]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1767916802,
 'time_last_update_utc': 'Fri, 09 Jan 2026 00:00:02 +0000',
 'time_next_update_unix': 1768003202,
 'time_next_update_utc': 'Sat, 10 Jan 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.986}

In [22]:
convert.invoke({'base_currency_value': 10, 'conversion_rate': 89.986})

899.86

In [24]:
# tool Binding
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=1
)

In [59]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [60]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you Convert 10 usd to inr')]

In [61]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you Convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [62]:
ai_message = llm_with_tools.invoke(messages)

In [63]:
ai_message

AIMessage(content='', additional_kwargs={'reasoning_content': 'The user wants conversion factor USD to INR and convert 10 USD to INR. We need to fetch conversion factor via get_conversion_factor. Then compute conversion via convert? Actually convert function expects base_currency_value, but we also need to specify currency? Possibly need to set context. The convert function maybe uses previously fetched factor? It says "Given a Currency Conversion Rate this function calculates the target currency value from a given base currency value". Probably we need to provide base_currency_value and maybe also the factor? But function signature only has base_currency_value. Might be that the function uses internal state of previous call? Not sure. Safer: we can just compute manually after getting factor.\n\nThus: call get_conversion_factor with base_currency USD, target_currency INR. Then compute 10 * factor.\n\nLet\'s call function.', 'tool_calls': [{'id': 'fc_b9cec962-3b7e-48ce-be13-d29f8908961b

In [64]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'fc_b9cec962-3b7e-48ce-be13-d29f8908961b',
  'type': 'tool_call'}]

In [67]:
for tool_call in ai_message.tool_calls:
    # Execute the 1st Tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)
        # Fetch this conversion rate
        print(tool_message1.content)
        print(tool_message1.content['conversion_rate'])
        # append this tool message to messages list
    # Execute the 2nd tool using the conversion rate from tool 1
 

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1767916802, "time_last_update_utc": "Fri, 09 Jan 2026 00:00:02 +0000", "time_next_update_unix": 1768003202, "time_next_update_utc": "Sat, 10 Jan 2026 00:00:02 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 89.986}' name='get_conversion_factor' tool_call_id='fc_b9cec962-3b7e-48ce-be13-d29f8908961b'
{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1767916802, "time_last_update_utc": "Fri, 09 Jan 2026 00:00:02 +0000", "time_next_update_unix": 1768003202, "time_next_update_utc": "Sat, 10 Jan 2026 00:00:02 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 89.986}


TypeError: string indices must be integers, not 'str'

In [ ]:
import json

for tool_call in ai_message.tool_calls:
    # Execute the 1st Tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)
        # Fetch this conversion rate
        print(tool_message1.content)
        print(tool_message1.content['conversion_rate'])
        # append this tool message to messages list
    # Execute the 2nd tool using the conversion rate from tool 1
 

KeyboardInterrupt: 